In [1]:
import numpy as np
import tensornetwork as tn

In [2]:
nCX = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
]).reshape(2, 2, 2, 2)

CX = tn.Node(nCX)

In [3]:
def test(P):    # a_in, b_in, a_out, b_out
    P = P.copy()
    ZERO = tn.Node(np.array([1, 0]))
    P = tn.contract(P[1] ^ ZERO[0])
    t = P.tensor.reshape(2, 4)
    return t[0] - t[1]

def untangle(P, T):
    T = tn.Node(np.array(T).reshape(2, 4).T)
    P = P.copy()
    ZERO = tn.Node(np.array([1, 0]))
    P = tn.contract(P[1] ^ ZERO[0])
    P = tn.Node(P.tensor.reshape(2, 4))
    print("P:", P.tensor)
    print("T:", T.tensor)
    P = tn.contract(P[1] ^ T[0])
    return tn.Node(P.tensor.reshape(2, 2))
    

In [4]:
test(CX)
res = untangle(CX, [1, 0, 0, 1, 1, 0, 0, 1])
print(f"rank: {np.linalg.matrix_rank(res.tensor)}")

ZERO = tn.Node(np.array([1, 0]))
ONE = tn.Node(np.array([0, 1]))
print(tn.contract(ZERO[0] ^ res.copy()[0]).tensor)
print(tn.contract(ONE[0] ^ res.copy()[0]).tensor)

P: [[1 0 0 0]
 [0 0 0 1]]
T: [[1 1]
 [0 0]
 [0 0]
 [1 1]]
rank: 1
[1 1]
[1 1]


In [5]:
# Random 4x4 unitary matrix
U = np.random.rand(4, 4)
U, _ = np.linalg.qr(U)
U = U.round(2)

U = tn.Node(U.reshape(2, 2, 2, 2))
U4 = U.tensor.reshape(4, 4)
print(U4)
print(U4 @ U4.T)

[[-0.99  0.12 -0.03 -0.05]
 [-0.08 -0.73  0.49 -0.47]
 [-0.04 -0.57 -0.82  0.03]
 [-0.09 -0.36  0.29  0.88]]
[[ 9.9790e-01  4.0000e-04 -5.7000e-03 -6.8000e-03]
 [ 4.0000e-04  1.0003e+00  3.4000e-03 -1.5000e-03]
 [-5.7000e-03  3.4000e-03  9.9980e-01 -2.6000e-03]
 [-6.8000e-03 -1.5000e-03 -2.6000e-03  9.9620e-01]]


In [6]:
U = tn.Node(
    np.array([
        [-0.08, 0.14, 0.96, -0.23],
        [-0.75, -0.06, 0.11, 0.65],
        [-0.62, -0.28, -0.18, -0.71],
        [-0.22, 0.95, -0.19, -0.13]
    ]).reshape(2, 2, 2, 2)
)

In [7]:
print(test(U))
res = untangle(U, [2.5, 1, 1, -1.66, 2.5, 1, 1, -1.66])
print(f"rank: {np.linalg.matrix_rank(res.tensor)}")

ZERO = tn.Node(np.array([1, 0]))
ONE = tn.Node(np.array([0, 1]))
print(tn.contract(ZERO[0] ^ res.copy()[0]).tensor)
print(tn.contract(ONE[0] ^ res.copy()[0]).tensor)

[0.54 0.42 1.14 0.48]
P: [[-0.08  0.14  0.96 -0.23]
 [-0.62 -0.28 -0.18 -0.71]]
T: [[ 2.5   2.5 ]
 [ 1.    1.  ]
 [ 1.    1.  ]
 [-1.66 -1.66]]
rank: 1
[1.2818 1.2818]
[-0.8314 -0.8314]


In [111]:
P = U.copy()
Z = tn.Node(np.array([1, 0]))
T = tn.Node(np.array([0.6, -0.6, 0.85, -0.85, -0.85, 0.85, 0.6, -0.6]).reshape(2, 2, 2))
kd = tn.Node(np.array([1, 0, 0, 0, 0, 0, 0, 1]).reshape(2, 2, 2))
tn.connect(P[1], Z[0])
tn.connect(P[2], kd[0])
tn.connect(kd[1], T[0])
tn.connect(P[3], T[1])
R = tn.contractors.auto([P, Z, T, kd], output_edge_order=[P[0], kd[2], T[2]])
for i in range(8):
    print(f"{i:03b}  {R.tensor.flatten()[i]:.3f}")

s = np.linalg.svd(R.tensor.reshape(4, 2), compute_uv=False)
print(s.round(3))

000  0.071
001  -0.071
010  -0.954
011  0.954
100  -0.610
101  0.610
110  -0.273
111  0.273
[1.65 0.  ]


In [108]:
P = U.copy()
Z = tn.Node(np.array([1, 0]))
T = tn.Node(np.array([0.6, 0.85, -0.85, 0.6, -0.6, -0.85, 0.85, -0.6]).reshape(2, 2, 2))
tn.connect(P[1], Z[0])
tn.connect(P[2], T[1])
tn.connect(P[3], T[2])
R = tn.contractors.auto([P, Z, T], output_edge_order=[P[0], T[0]])
R.tensor.reshape(2, 2)

array([[-0.883,  0.883],
       [-0.883,  0.883]])

In [105]:
U = U.copy()
R = tn.contract(tn.connect(U[1], tn.Node(np.array([1, 0]))[0]))

In [106]:
R.tensor.shape  # (2, 2, 2)
t = []
for i in range(2):
    for j in range(2):
        print(f"{i} {j}   ", round(R.tensor[i, j, 0] ** 2 + R.tensor[i, j, 1] ** 2, 2))


0 0    0.03
0 1    0.97
1 0    0.46
1 1    0.54


In [113]:
t = np.array([-0.071, 0.954, 0.610, 0.273])
t = t ** 2
t = t / sum(np.array([-0.071, 0.954]) ** 2)
t

array([0.00550834, 0.99449166, 0.4065969 , 0.08143849])

In [101]:
np.array([0.54, 0.42, 1.14, 0.48]) @ np.array([0.6, 0.85, -0.85, 0.6])

np.float64(4.44089209850063e-19)